<a href="https://colab.research.google.com/github/BI-DS/ELE-3909/blob/master/lecture11/apriori_algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import mlxtend
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
import warnings
warnings.filterwarnings('ignore')

# The apriori algorithm on a transaction database

In [3]:
# Download dataset
path = 'https://raw.githubusercontent.com/BI-DS/ELE-3909/refs/heads/master/lecture11/Retail.csv'
online = pd.read_csv(path)
online.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


## Data cleaning and formating

a) Create an indicator variable stipulating whether the invoice number begings with "C"

In [4]:
online['IsCPresent'] = (
    online['InvoiceNo']
    .astype(str)
    .apply(lambda x: 1 if x.find('C') != -1 else 0)
)

b) Filter out all transactions having either zero or negative number of items, remove all invoice numbers starting with "C", subset the DataFrame down to `InvoiceNo` and `Description`, and drop all rows with at least one missing value.

In [5]:
online1 = (
    online
    # filter out non-positive quantity values
    .loc[online["Quantity"] > 0]
    # remove InvoiceNos starting with C
    .loc[online['IsCPresent'] != 1]
    # column filtering
    .loc[:, ["InvoiceNo", "Description"]]
    # dropping all rows with at least one missing value
    .dropna()
)
online1.head()

,InvoiceNo,Description
0,536365,WHITE HANGING HEART T-LIGHT HOLDER
1,536365,WHITE METAL LANTERN
2,536365,CREAM CUPID HEARTS COAT HANGER
3,536365,KNITTED UNION FLAG HOT WATER BOTTLE
4,536365,RED WOOLLY HOTTIE WHITE HEART.


c) Extract the invoice numbers from the DataFrame as a list. Remove duplicates to create a list of unique invoice numbers

In [6]:
invoice_no_list = online1.InvoiceNo.tolist()
invoice_no_list = list(set(invoice_no_list))

d) Select the first 5,000 elements of the list

In [7]:
subset_invoice_no_list = invoice_no_list[0:5000]
online1 = online1.loc[online1["InvoiceNo"].isin(subset_invoice_no_list)]
online1.head(10)

,InvoiceNo,Description
142,536382,INFLATABLE POLITICAL GLOBE
143,536382,VINTAGE SNAKES & LADDERS
144,536382,CHOCOLATE CALCULATOR
145,536382,JUMBO SHOPPER VINTAGE RED PAISLEY
146,536382,RECYCLING BAG RETROSPOT
147,536382,TOY TIDY PINK POLKADOT
148,536382,ANTIQUE GLASS DRESSING TABLE POT
149,536382,ALARM CLOCK BAKELIKE GREEN
150,536382,IVORY GIANT GARDEN THERMOMETER
151,536382,3 TIER CAKE TIN GREEN AND CREAM


e) Iterate over the unique invoice numbers and extract the item descriptions as a list and append that list to the larger `invoice_item_list`

In [8]:
invoice_item_list = []
for num in list(set(online1.InvoiceNo.tolist())):
    # filter data set down to one invoice number
    tmp_df = online1.loc[online1['InvoiceNo'] == num]
    # extract item descriptions and convert to list
    tmp_items = tmp_df.Description.tolist()
    # append list invoice_item_list
    invoice_item_list.append(tmp_items)

print(invoice_item_list[0])

['DOORMAT NEW ENGLAND', 'HOMEMADE JAM SCENTED CANDLES', 'JAM MAKING SET WITH JARS', 'JAM MAKING SET PRINTED', 'OFFICE MUG WARMER POLKADOT', 'OFFICE MUG WARMER CHOC+BLUE', 'WHITE SKULL HOT WATER BOTTLE ', 'GREY HEART HOT WATER BOTTLE', 'JUMBO BAG RED RETROSPOT', 'JUMBO BAG SPACEBOY DESIGN', 'DELUXE SEWING KIT ', 'BLUE POLKADOT WRAP', 'WHITE HANGING HEART T-LIGHT HOLDER']


f) Encode the invoice item list above into logical vectors

In [9]:
online_encoder = TransactionEncoder()
online_encoder_array = online_encoder.fit_transform(invoice_item_list)
print(online_encoder_array)

[[False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]]


g) Create a DataFrame of he logical vectors including the product names

In [10]:
online_encoder_df = pd.DataFrame(
    online_encoder_array,
    columns=online_encoder.columns_
)

# this is a very big table, so for more
# easy viewing only a subset is printed
online_encoder_df.loc[
    4970:4979,
    online_encoder_df.columns.tolist()[0:8]
]

,4 PURPLE FLOCK DINNER CANDLES,50'S CHRISTMAS GIFT BAG LARGE,DOLLY GIRL BEAKER,I LOVE LONDON MINI BACKPACK,NINE DRAWER OFFICE TIDY,OVAL WALL MIRROR DIAMANTE,RED SPOT GIFT BAG LARGE,SET 2 TEA TOWELS I LOVE LONDON
4970,False,False,False,False,False,False,False,False
4971,False,False,False,False,False,False,False,False
4972,False,False,False,False,False,False,False,False
4973,False,False,False,False,False,False,False,False
4974,False,False,False,False,False,False,False,False
4975,False,False,False,False,False,False,False,False
4976,False,False,False,False,False,False,False,False
4977,False,False,False,False,False,False,False,False
4978,False,False,False,False,False,False,False,False
4979,False,False,True,False,False,False,False,False


## Apriori Algorithm

Run the apriori algorithm with a minimum support of 0.01

In [11]:
mod_colnames_minsupport = apriori(
    online_encoder_df,
    min_support=0.01,
    use_colnames=True
)
mod_colnames_minsupport.loc[0:6]

,support,itemsets
0,0.0118,( SET 2 TEA TOWELS I LOVE LONDON )
1,0.0168,(10 COLOUR SPACEBOY PEN)
2,0.0132,(12 MESSAGE CARDS WITH ENVELOPES)
3,0.0174,(12 PENCIL SMALL TUBE WOODLAND)
4,0.0172,(12 PENCILS SMALL TUBE RED RETROSPOT)
5,0.0172,(12 PENCILS SMALL TUBE SKULL)
6,0.0128,(12 PENCILS TALL TUBE RED RETROSPOT)


Calculate the length of the item sets

In [12]:
mod_colnames_minsupport['length'] = (
    mod_colnames_minsupport['itemsets'].apply(lambda x: len(x))
)

After we have calculated the length of all item sets, we can query transaction with mininum support of 0.01 and min length of 2, for example

In [16]:
mod_colnames_minsupport[
    (mod_colnames_minsupport['length'] >= 3) &
    (mod_colnames_minsupport['support'] >= 0.01)
]

,support,itemsets,length
1732,0.0102,"(PAPER CHAIN KIT 50'S CHRISTMAS , PAPER CHAIN ...",3
1733,0.0100,"(60 TEATIME FAIRY CAKE CASES, PACK OF 72 RETRO...",3
1734,0.0108,"(ALARM CLOCK BAKELIKE RED , ALARM CLOCK BAKELI...",3
1735,0.0124,"(ALARM CLOCK BAKELIKE IVORY, ALARM CLOCK BAKEL...",3
1736,0.0106,"(ALARM CLOCK BAKELIKE RED , ALARM CLOCK BAKELI...",3
...,...,...,...
2092,0.0100,"(HERB MARKER THYME, HERB MARKER CHIVES , HERB ...",5
2093,0.0106,"(HERB MARKER THYME, HERB MARKER PARSLEY, HERB ...",5
2094,0.0106,"(HERB MARKER THYME, HERB MARKER PARSLEY, HERB ...",5
2095,0.0108,"(HERB MARKER THYME, HERB MARKER PARSLEY, HERB ...",5
